In [2]:
import pandas as pd
import visual_behavior.database as db
from visual_behavior.visualization.qc.dash_app import functions

In [5]:
def does_session_have_qc(session_id, attribute):
    
    conn = db.Database('visual_behavior_data')
    collection = conn['ophys_qc']['session_qc_records']
    res = list(collection.find({'session_id':session_id, 'qc_attribute':attribute}))
    conn.close()
    
    return len(res) > 0

def mark_session_as_qcd(session_id, attribute):
    entry = {
        'session_id':session_id,
        attribute:True
    }
    
    conn = db.Database('visual_behavior_data')
    collection = conn['ophys_qc']['session_qc']
    db.update_or_create(collection, db.clean_and_timestamp(entry), keys_to_check=['session_id'])
    conn.close()

In [6]:
session_id = 1048363441
attribute = 'Decrosstalking - Session Level'
does_session_have_qc(session_id, attribute)

True

In [7]:
session_id = 1048363441

conn = db.Database('visual_behavior_data')
collection = conn['ophys_qc']['session_qc_records']
res = list(collection.find({'session_id':int(session_id)}))
conn.close()

print(res)

[{'_id': '80992a7c19cc4fd88505cba1461dc57b', 'timestamp': '2021-01-20 17:45:20', 'username': 'test_dougo', 'session_id': 1048363441, 'experiment_id': 1048483604, 'qc_attribute': 'Decrosstalking - Session Level', 'motion_present': None, 'qc_labels': 'no_problems', 'input_text': '', 'entry_time_utc': '2021-01-21 01:45:52.270178'}, {'_id': '1d718bb1a399456ca5dd78108d96d1ba', 'timestamp': '2021-01-20 17:50:28', 'username': 'test_doug', 'session_id': 1048363441, 'experiment_id': 1048483604, 'qc_attribute': 'Decrosstalking - Session Level', 'motion_present': None, 'qc_labels': 'no_problems', 'input_text': 'test', 'entry_time_utc': '2021-01-21 01:50:41.986721'}, {'_id': 'c39c7eb6649f4450babd906acbe58508', 'timestamp': '2021-01-20 17:51:25', 'username': 'test_doug', 'session_id': 1048363441, 'experiment_id': 1048483604, 'qc_attribute': 'Decrosstalking - Session Level', 'motion_present': None, 'qc_labels': 'no_problems', 'input_text': '', 'entry_time_utc': '2021-01-21 01:51:31.734318'}]


In [8]:
res

[{'_id': '80992a7c19cc4fd88505cba1461dc57b',
  'timestamp': '2021-01-20 17:45:20',
  'username': 'test_dougo',
  'session_id': 1048363441,
  'experiment_id': 1048483604,
  'qc_attribute': 'Decrosstalking - Session Level',
  'motion_present': None,
  'qc_labels': 'no_problems',
  'input_text': '',
  'entry_time_utc': '2021-01-21 01:45:52.270178'},
 {'_id': '1d718bb1a399456ca5dd78108d96d1ba',
  'timestamp': '2021-01-20 17:50:28',
  'username': 'test_doug',
  'session_id': 1048363441,
  'experiment_id': 1048483604,
  'qc_attribute': 'Decrosstalking - Session Level',
  'motion_present': None,
  'qc_labels': 'no_problems',
  'input_text': 'test',
  'entry_time_utc': '2021-01-21 01:50:41.986721'},
 {'_id': 'c39c7eb6649f4450babd906acbe58508',
  'timestamp': '2021-01-20 17:51:25',
  'username': 'test_doug',
  'session_id': 1048363441,
  'experiment_id': 1048483604,
  'qc_attribute': 'Decrosstalking - Session Level',
  'motion_present': None,
  'qc_labels': 'no_problems',
  'input_text': '',
  

In [9]:
entry = {
    'session_id':session_id,
    attribute:True
}
entry

{'session_id': 1048363441, 'Decrosstalking - Session Level': True}

In [10]:
mark_session_as_qcd(session_id, attribute)

In [12]:
session_table = functions.load_session_data()
attribute = 'Decrosstalking - Session Level'
for idx,row in session_table.iterrows():
    osid = int(row['ophys_session_id'])
    print(osid, does_session_have_qc(osid, attribute))
    if does_session_have_qc(osid, attribute):
        mark_session_as_qcd(osid, attribute)

Getting all ophys sessions. This might take a while.


952430817 True
955775716 True
958105827 False
958772311 True
959458018 True
988768058 False
989267296 False
1048363441 True
1049240847 True
1050231786 False
1051319542 False
941676716 True
929686773 False
931326814 False
935559843 False
937162622 True
938140092 False
939526443 False
1052752249 False
882386411 False
848983781 False
881094781 False
882060185 False
882674040 False
884451806 False
886130638 False
886806800 False
889944877 False
944888114 False
947199653 False
948042811 False
949217880 False
870352564 False
871526950 False
872592724 False
873247524 False
973384292 True
974167263 False
974486549 False
904771513 False
866197765 True
867027875 False
868688430 False
870762788 False
976382032 True
986130604 False
917498735 False
919888953 False
920695792 True
849304162 False
852794141 False
993228542 False
971849979 False
1040797492 False
1041082185 False
994136636 False
993942673 False
993504279 False
1038963428 False
1039252364 False
959735908 False
960606804 False
962045676 F

In [18]:
%%time
def update_session_table(session_table):
    for idx,row in session_table.iterrows():
        if does_session_have_qc(osid, attribute):
            session_table.at[idx,'has_decrosstalk_qc'] = True
    return session_table

CPU times: user 9 µs, sys: 3 µs, total: 12 µs
Wall time: 21.5 µs


In [20]:
import visual_behavior.database as db
import pandas as pd
def get_session_qc_records(search_dict={}):
    '''
    find all session level qc records matching the pattern in search_dict
    passing an empty search dict (or no args) returns all records
    returns pandas dataframe
    '''
    conn = db.Database('visual_behavior_data')
    collection = conn['ophys_qc']['session_qc_records']
    res = list(collection.find(search_dict))
    conn.close()
    return pd.DataFrame(res)

In [22]:
get_session_records({'username':'natalia'})

,_id,timestamp,username,session_id,experiment_id,qc_attribute,motion_present,qc_labels,input_text,entry_time_utc
0,97b117878486409fa4fc15149b7a274f,2021-01-21 11:12:12,natalia,952430817,953659749,Decrosstalking - Session Level,None,overlap_plots_missing,13,2021-01-21 19:12:31.843178
1,c7b79b7d6d954091a45ee74dd71478aa,2021-01-21 11:15:20,natalia,952430817,953659749,Decrosstalking - Session Level,None,outlier_no_double,19,2021-01-21 19:15:40.140500
2,9ba93866e7e244c9aab94427098590bc,2021-01-21 11:20:55,natalia,952430817,953659752,Decrosstalking - Session Level,None,overlap_plots_missing,14,2021-01-21 19:21:26.863718
3,77a3e5e77abe475e9be23a98d5cdc8da,2021-01-21 11:23:48,natalia,952430817,953659756,Decrosstalking - Session Level,None,ghost_ROI_has_no_double,3,2021-01-21 19:24:11.258393
4,044ab5c647ef4c709e6671661b14069f,2021-01-21 11:46:40,natalia,976382032,977247470,Decrosstalking - Session Level,None,too_many_outlier_ROIs,,2021-01-21 19:46:52.756509
5,d289f646a3a54b57abbd73e83f183e03,2021-01-21 11:51:59,natalia,976382032,977247470,Decrosstalking - Session Level,None,outlier_not_ghost,12,2021-01-21 19:52:22.215952
6,9d96cbc5e1c6450f967cf0819844ebe1,2021-01-21 12:15:28,natalia,952430817,953659743,Decrosstalking - Session Level,None,missed_ghost,4,2021-01-21 20:15:36.884226
7,feed9ce7a5344725ae3be90395c2a86f,2021-01-21 12:42:23,natalia,989267296,989610989,Decrosstalking - Session Level,None,overlap_plots_missing,,2021-01-21 20:42:52.980391
